In [ ]:
from sage.all import *
from math import log, floor
from random import randint
import multiprocessing
from math import log, floor, sqrt, exp

def LpCalculating(p):
    log_p = log(p)
    log_log_p = log(log_p)
    return exp((1 / sqrt(2)) * sqrt(log_p) * sqrt(log_log_p))

def lenstra_ecm(N, B1=50, B2=200, maxCurves=300):
    for i in range(maxCurves):
        print(f"\nПопытка {i}:")
        #print(f"try {i}")
        x = randint(1, N - 1)
        y = randint(1, N - 1)
        a = randint(0, N - 1)
        b = (y**2 - x**3 - a*x) % N

        discr = (4*a**3 + 27*b**2) % N
        g = gcd(discr, N)
        if g == N:
            continue
        if 1 < g < N:
            return g

        try:
            curve = EllipticCurve(Zmod(N), [a, b])
            P = curve(x, y)
        except (ArithmeticError, ValueError):
            continue

        for p in prime_range(2, B1 + 1):
            e = floor(log(B2) / log(p))
            k = p^e
            try:
                R = k * P
            except ZeroDivisionError:
                try:
                    g = gcd(P[0].lift(), N)
                except Exception:
                    g = 1
                if 1 < g < N:
                    return g
                else:
                    continue
            except (ArithmeticError, ValueError):
                continue
    return None


# Обёртка для multiprocessing
def attempt_curve(args):
    N, B1, B2, maxCurves = args
    return lenstra_ecm(N, B1, B2, maxCurves)

if __name__ == "__main__":
    # Генерация N = p * q
    p = random_prime(10**5)
    q = random_prime(10**12)
    N = p * q
    print(f"p = {p} (битность: {p.nbits()})")
    print(f"q = {q} (битность: {q.nbits()})")
    print(f"N = {N} (битность: {N.nbits()})")

    Lp = LpCalculating(p)
    print(f"L_p({p}) = {Lp}")
    B1 = int(Lp)
    print(f"B1 = {B1}")
    mulNumber = 5
    B2 = mulNumber * B1
    MAX_CURVES_PER_WORKER = 30000
    NUM_PROCESSES = multiprocessing.cpu_count()
    TOTAL_TRIES = 10000

    print(f"\nЗапускаем ECM на {8} ядрах…")

    with multiprocessing.Pool(processes=8) as pool:
        args = [(N, B1, B2, 4)] * TOTAL_TRIES
        for result in pool.imap_unordered(attempt_curve, args):
            if result:
                print(f"\nДелитель найден: {result}, второй: {N // result}")
                pool.terminate()
                break
        else:
            print("\nДелитель не найден.")